In [ ]:
# --- Headless Chrome setup that works on GitHub Actions and locally ---
import os, shutil
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import logging

# Configure logging once
logging.basicConfig(
    level=logging.INFO,  # change to DEBUG if you want more detail
    format="%(asctime)s [%(levelname)s] %(message)s"
)

logger = logging.getLogger(__name__)

opts = Options()
opts.add_argument("--headless=new")          # required on CI
opts.add_argument("--no-sandbox")            # required on CI
opts.add_argument("--disable-dev-shm-usage") # required on CI
opts.add_argument("--disable-gpu")
# unique profile per run to avoid "user data directory is already in use"
opts.add_argument(f"--user-data-dir=/tmp/chrome-profile-{os.getenv('RUN_ID','local')}")

# Use chromedriver installed on the runner (if present), else let Selenium manage locally
drv_path = shutil.which("chromedriver")
service = Service(drv_path) if drv_path else None
browser = webdriver.Chrome(service=service, options=opts)




In [ ]:
## Packages

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import glob
import os
import time
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [ ]:
## Functions
summary_ = None
#Does the variable exist (by xpath)
def check_exists_by_xpath(xpath):
    time.sleep(1)
    try:
        browser.find_element("xpath",xpath)
    except NoSuchElementException:
        return False
    return True

#Does the variable exist (by name)
def CheckExistsByName(find_name):
    time.sleep(1)
    try:
        browser.find_element("name",find_name)
    except NoSuchElementException:
        return False
    return True

#Scrape data from day summary page
def summary_page_scrape(input_date):
    row_list = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]
    case_type_xpath = []
    case_defendant_xpath = []
    case_plaintiff_xpath = []
    case_number_xpath = []
    case_hearing_time_xpath = []
    for i in row_list:
        if check_exists_by_xpath("/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/table/tbody/tr[3]/td/table/tbody/tr["+ str(i) +"]/td[5]") == True:
            case_type_xpath.append("/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/table/tbody/tr[3]/td/table/tbody/tr["+ str(i) +"]/td[5]")
            case_defendant_xpath.append("/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/table/tbody/tr[3]/td/table/tbody/tr["+ str(i) +"]/td[3]")
            case_plaintiff_xpath.append("/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/table/tbody/tr[3]/td/table/tbody/tr["+ str(i) +"]/td[4]")
            case_hearing_time_xpath.append("/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/table/tbody/tr[3]/td/table/tbody/tr["+ str(i) +"]/td[6]")
            case_number_xpath.append("/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/table/tbody/tr[3]/td/table/tbody/tr["+ str(i) +"]/td[2]/a")
    case_type = []
    for case in case_type_xpath:
        case_type_i = browser.find_element("xpath",case).text
        case_type.append(case_type_i)
    case_number = []
    for case in case_number_xpath:
        case_number_i = browser.find_element("xpath",case).text
        case_number.append(case_number_i)
    case_defendant = []
    for case in case_defendant_xpath:
        case_defendant_i = browser.find_element("xpath",case).text
        case_defendant.append(case_defendant_i)
    case_plaintiff = []
    for case in case_plaintiff_xpath:
        case_plaintiff_i = browser.find_element("xpath",case).text
        case_plaintiff.append(case_plaintiff_i)
    case_hearing_time = []
    for case in case_hearing_time_xpath:
        case_hearing_time_i = browser.find_element("xpath",case).text
        case_hearing_time.append(case_hearing_time_i)
    d = {'CaseNumber':case_number,'Defendant':case_defendant,'Plaintiff':case_plaintiff,'CaseType':case_type, 'HearingTime':case_hearing_time}
    df = pd.DataFrame(d)
    df['HearingDate'] = input_date
    input_date_clean = input_date.replace("/","-")
    df.to_csv('summary_'+input_date_clean+"_"+ str(k) +".csv", index = False)  
    summary_ = df

def deletefiles(path):
    # files_in_directory = os.listdir(path)
    # filtered_files = [file for file in files_in_directory if file.endswith(".csv")]
    # for file in filtered_files:
    #     path_to_file = os.path.join(path, file)
    #     os.remove(path_to_file)
    logger.info("Will not delete, haha!")
    

In [ ]:
path = r'C:/Users/katherine.key/OneDrive - City of Alexandria/Documents/Evictions/Data/Raw/scraper_test/' 
deletefiles(path)

#browser = webdriver.Chrome()
logger.info("Started")

browser.get('https://eapps.courts.state.va.us/gdcourts')
time.sleep(3)
#Accept the Terms and Conditions of Use
elem = browser.find_element("xpath",'/html/body/table/tbody/tr[1]/td/table/tbody/tr[3]/td/form/table/tbody/tr[2]/td/input[1]').click()  # Find the search box
time.sleep(3)

#Manual date(s) set if needed - code below automatically searches previous six weeks and six weeks forward 
input_dates = []

curr_date = pd.to_datetime('today').date()

start_date = curr_date - relativedelta(days = 84)
#start = datetime.datetime.strptime(start_date, "%d-%m-%Y")
date_generated = pd.date_range(start_date, periods = 140)
#date_generated = pd.date_range(start='06/29/2022', end='06/30/2022')


input_dates = list(date_generated.strftime("%m/%d/%Y"))

print(input_dates)
logger.info(input_dates)

for input_date in input_dates:
    #Go to General Court of Alexandria
    browser.get('https://eapps.courts.state.va.us/gdcourts/caseSearch.do?fromSidebar=true&searchLanding=searchLanding&searchType=hearingDate&searchDivision=V&searchFipsCode=510&curentFipsCode=510')
    #time.sleep(3)
    browser.find_element("xpath",'/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/table/tbody/tr/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td[2]/input').send_keys(input_date)
    search = browser.find_element("xpath",'/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/table/tbody/tr/td/table/tbody/tr[3]/td/input[1]').click()

    k = 1
    
    while CheckExistsByName("caseInfoScrollForward") == True:
        summary_page_scrape(input_date)
        k = k+1
        browser.find_element("name","caseInfoScrollForward").click()
        logger.info('still inside the while loop')
    else:
        summary_page_scrape(input_date)
    

['06/21/2025', '06/22/2025', '06/23/2025', '06/24/2025', '06/25/2025', '06/26/2025', '06/27/2025', '06/28/2025', '06/29/2025', '06/30/2025', '07/01/2025', '07/02/2025', '07/03/2025', '07/04/2025', '07/05/2025', '07/06/2025', '07/07/2025', '07/08/2025', '07/09/2025', '07/10/2025', '07/11/2025', '07/12/2025', '07/13/2025', '07/14/2025', '07/15/2025', '07/16/2025', '07/17/2025', '07/18/2025', '07/19/2025', '07/20/2025', '07/21/2025', '07/22/2025', '07/23/2025', '07/24/2025', '07/25/2025', '07/26/2025', '07/27/2025', '07/28/2025', '07/29/2025', '07/30/2025', '07/31/2025', '08/01/2025', '08/02/2025', '08/03/2025', '08/04/2025', '08/05/2025', '08/06/2025', '08/07/2025', '08/08/2025', '08/09/2025', '08/10/2025', '08/11/2025', '08/12/2025', '08/13/2025', '08/14/2025', '08/15/2025', '08/16/2025', '08/17/2025', '08/18/2025', '08/19/2025', '08/20/2025', '08/21/2025', '08/22/2025', '08/23/2025', '08/24/2025', '08/25/2025', '08/26/2025', '08/27/2025', '08/28/2025', '08/29/2025', '08/30/2025', '08/3

In [ ]:
# Read in files to combine into one data frame
# Code below to delete the temporary files in the folder


path = r'C:/Users/katherine.key/OneDrive - City of Alexandria/Documents/Evictions/Data/Raw/scraper_test/' # use your path
#all_files = glob.glob(path + "/*.csv")
all_files = summary_

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)


In [5]:
# Filter to just Unlawful Detainers
UD_days = frame[frame.CaseType.str.contains('Unlawful')]

# Number of hearings for the date range
len(UD_days.index)
#print(UD_days)

1677

In [ ]:
# Delete page files at the end of the code (to make sure we are storing hundreds of csv's)
path = r'C:/Users/ine.key/OneDrive - City of Alexandria/Documents/Evictions/Data/Raw/scraper_test/' 
deletefiles(path)

In [9]:
# Scrape information from individual case number pages

# Go to Case Number Search (Set to Alexandria FIPS)
browser.get('https://eapps.courts.state.va.us/gdcourts/criminalCivilCaseSearch.do?fromSidebar=true&formAction=searchLanding&searchDivision=V&searchFipsCode=510&curentFipsCode=510')

# Convert Case Numbers from UD_days to List for Case Number iteration search 
case_num_list_repeat = UD_days['CaseNumber'].tolist()
# Make the list unique in case there are repeat case numbers
case_num_array = np.array(case_num_list_repeat)
case_num_array_dist = np.unique(case_num_array)
case_num_list = case_num_array_dist.tolist()
#case_num_list.remove('GV22000345-00') #Commercial case with mixed claims that I didn't want to have to write a different case for
#case_num_list.remove('GV22001032-00') #I assume an expunged case - need to figure out how to write this. 
#case_num_list.remove('GV22005997-00') #Appealed to circuit court
#case_num_list.remove('GV23005068-00') #Appealed to circuit court
#case_num_list.remove('GV23005179-00') #Appealed to circuit court
#case_num_list.remove('GV23005439-00') #Appealed to circuit court
#case_num_list.remove('GV23005440-00') #Appealed to circuit court
#case_num_list.remove('GV23005862-00') #Appealed to circuit court
#case_num_list.remove('GV23005210-00') #Appealed to circuit court
#case_num_list.remove('GV23005211-00') #Appealed to circuit court
#case_num_list.remove('GV23005864-00') #Appealed to circuit court
#case_num_list.remove('GV23005865-00') #Appealed to circuit court
#case_num_list.remove('GV24000847-00') #Appealed to circuit court
#case_num_list.remove('GV24003249-00') #Appealed to circuit court
#case_num_list.remove('GV24003253-00') #Appealed to circuit court
#case_num_list.remove('GV24004676-00') #Appealed to circuit court
#case_num_list.remove('GV24004997-00') #Appealed to circuit court
#case_num_list.remove('GV24005624-00') #Appealed to circuit court
#case_num_list.remove('GV24007477-00') #Appealed to circuit court
#case_num_list.remove('GV25001187-00') #Appealed to circuit court
#case_num_list.remove('GV25001272-00') #Appealed to circuit court
#case_num_list.remove('GV25002232-00') #Appealed to circuit court
case_num_list.remove('GV25001440-00') #Appealed to circuit court
case_num_list.remove('GV25004755-00') #Appealed to circuit court
case_num_list.remove('GV25004708-00') #Appealed to circuit court
case_num_list.remove('GV25005308-00') #Appealed to circuit court




# Initialize all variables on website CaseNumber - probably want this outside of the loop
CaseNumber = []
FiledDate = []
CaseType = []
DebtType = []
Judgment = []
Costs = []
AttorneyFees = []
PrincipalAmount = []
OtherAmount = []
InterestAward = []
Possession = []
WritIssuedDate = []
HomesteadExemptionWaived = []
IsJudgmentSatisfied = []
DateSatisfactionFiled = []
OtherAwarded = []
FurtherCaseInformation = []
Garnishee = []
Address = []
GarnisheeAnswer = []
AnswerDate = []
NumberofChecksReceived = []
AppealDate = []
AppealedBy = []
WritofEvictionIssuedDate = []
WritofFieriFaciasIssuedDate = []
Plaintiff1Name = []
Plaintiff1DBATA = []
Plaintiff1Address = []
Plaintiff1Attorney = []
Plaintiff2Name = []
Plaintiff2DBATA = []
Plaintiff2Address = []
Plaintiff2Attorney = []
Plaintiff3Name = []
Plaintiff3DBATA = []
Plaintiff3Address = []
Plaintiff3Attorney = []
Defendant1Name = []
Defendant1DBATA = []
Defendant1Address = []
Defendant1Attorney = []
Defendant2Name = []
Defendant2DBATA = []
Defendant2Address = []
Defendant2Attorney = []
Defendant3Name = []
Defendant3DBATA = []
Defendant3Address = []
Defendant3Attorney = []
Defendant4Name = []
Defendant4DBATA = []
Defendant4Address = []
Defendant4Attorney = []
Defendant5Name = []
Defendant5DBATA = []
Defendant5Address = []
Defendant5Attorney = []
Defendant6Name = []
Defendant6DBATA = []
Defendant6Address = []
Defendant6Attorney = []
Next_Hearing_Date = []
Next_Hearing_Time = []
Admin_tag = []

for case_number in case_num_list:
    # detail_page_scrape(case_number)
    CaseNumber_i = ""
    FiledDate_i = ""
    CaseType_i = ""
    DebtType_i = ""
    Judgment_i = ""
    Costs_i = ""
    AttorneyFees_i = ""
    PrincipalAmount_i = ""
    OtherAmount_i = ""
    InterestAward_i = ""
    Possession_i = ""
    WritIssuedDate_i = ""
    HomesteadExemptionWaived_i = ""
    IsJudgmentSatisfied_i = ""
    DateSatisfactionFiled_i = ""
    OtherAwarded_i = ""
    FurtherCaseInformation_i = ""
    Garnishee_i = ""
    Address_i = ""
    GarnisheeAnswer_i = ""
    AnswerDate_i = ""
    NumberofChecksReceived_i = ""
    AppealDate_i = ""
    AppealedBy_i = ""
    WritofEvictionIssuedDate_i = ""
    WritofFieriFaciasIssuedDate_i = ""
    Plaintiff1Name_i = ""
    Plaintiff1DBATA_i = ""
    Plaintiff1Address_i = ""
    Plaintiff1Attorney_i = ""
    Plaintiff2Name_i = ""
    Plaintiff2DBATA_i = ""
    Plaintiff2Address_i = ""
    Plaintiff2Attorney_i = ""
    Plaintiff3Name_i = ""
    Plaintiff3DBATA_i = ""
    Plaintiff3Address_i = ""
    Plaintiff3Attorney_i = ""
    Defendant1Name_i = ""
    Defendant1DBATA_i = ""
    Defendant1Address_i = ""
    Defendant1Attorney_i = ""
    Defendant2Name_i = ""
    Defendant2DBATA_i = ""
    Defendant2Address_i = ""
    Defendant2Attorney_i = ""
    Defendant3Name_i = ""
    Defendant3DBATA_i = ""
    Defendant3Address_i = ""
    Defendant3Attorney_i = ""
    Defendant4Name_i = ""
    Defendant4DBATA_i = ""
    Defendant4Address_i = ""
    Defendant4Attorney_i = ""
    Defendant5Name_i = ""
    Defendant5DBATA_i = ""
    Defendant5Address_i = ""
    Defendant5Attorney_i = ""
    Defendant6Name_i = ""
    Defendant6DBATA_i = ""
    Defendant6Address_i = ""
    Defendant6Attorney_i = ""
    
    # Go to Case Number Search (Set to Alexandria FIPS)
    browser.get('https://eapps.courts.state.va.us/gdcourts/criminalCivilCaseSearch.do?fromSidebar=true&formAction=searchLanding&searchDivision=V&searchFipsCode=510&curentFipsCode=510')
    # Search case number
    browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/table/tbody/tr[2]/td/table/tbody/tr[1]/td[2]/input").send_keys(case_number)
    search = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/table/tbody/tr[3]/td/input[2]").click()

    if check_exists_by_xpath("/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/table/tbody/tr[4]/td/div") == True:
        CaseNumber_i = ""
    else:    
        if check_exists_by_xpath("/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[2]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td[2]") == True:
            CaseNumber_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[2]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td[2]").text
            FiledDate_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[2]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td[4]").text
            CaseType_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[2]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td[2]").text
            DebtType_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[2]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td[4]").text
            Judgment_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[14]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td[2]").text
            Costs_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[14]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td[4]").text
            AttorneyFees_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[14]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td[6]").text
            PrincipalAmount_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[14]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td[2]").text
            OtherAmount_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[14]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td[4]").text
            InterestAward_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[14]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td[6]").text
            Possession_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[14]/td/table/tbody/tr[2]/td/table/tbody/tr[3]/td[2]").text
            # WritIssuedDate may be redundant with WritofEvictionIssuedDate - right now just pulling the same information into both
            WritIssuedDate_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[14]/td/table/tbody/tr[2]/td/table/tbody/tr[3]/td[4]").text
            HomesteadExemptionWaived_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[14]/td/table/tbody/tr[2]/td/table/tbody/tr[4]/td[2]").text
            IsJudgmentSatisfied_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[14]/td/table/tbody/tr[2]/td/table/tbody/tr[5]/td[2]").text
            DateSatisfactionFiled_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[14]/td/table/tbody/tr[2]/td/table/tbody/tr[5]/td[4]").text
            OtherAwarded_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[14]/td/table/tbody/tr[2]/td/table/tbody/tr[5]/td[6]").text
            FurtherCaseInformation_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[14]/td/table/tbody/tr[2]/td/table/tbody/tr[6]/td[2]").text
            Garnishee_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[16]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td[2]").text
            Address_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[16]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td[4]").text
            GarnisheeAnswer_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[16]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td[2]").text
            AnswerDate_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[16]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td[3]/table/tbody/tr/td[2]").text
            NumberofChecksReceived_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[16]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td[3]/table/tbody/tr/td[4]").text
            AppealDate_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[18]/td/table/tbody/tr[2]/td/table/tbody/tr/td[2]").text
            AppealedBy_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[18]/td/table/tbody/tr[2]/td/table/tbody/tr/td[4]").text
            WritofEvictionIssuedDate_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[14]/td/table/tbody/tr[2]/td/table/tbody/tr[3]/td[4]").text
            WritofFieriFaciasIssuedDate_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[14]/td/table/tbody/tr[2]/td/table/tbody/tr[3]/td[6]").text


            Plaintiff1Name_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[4]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[2]/td[1]").text
            Plaintiff1DBATA_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[4]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[2]/td[2]").text
            Plaintiff1Address_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[4]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[2]/td[3]").text
            Plaintiff1Attorney_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[4]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[2]/td[5]").text

            # Had to wrap additional plaintiffs and defendents in if statements to see if the information exists
            if check_exists_by_xpath("/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[4]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[3]/td[1]") == True:
                Plaintiff2Name_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[4]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[3]/td[1]").text
                Plaintiff2DBATA_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[4]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[3]/td[2]").text
                Plaintiff2Address_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[4]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[3]/td[3]").text
                Plaintiff2Attorney_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[4]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[3]/td[5]").text

            if check_exists_by_xpath("/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[4]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[4]/td[1]") == True:
                Plaintiff3Name_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[4]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[4]/td[1]").text
                Plaintiff3DBATA_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[4]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[4]/td[2]").text
                Plaintiff3Address_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[4]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[4]/td[3]").text
                Plaintiff3Attorney_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[4]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[4]/td[5]").text

            Defendant1Name_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[2]/td[1]").text
            Defendant1DBATA_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[2]/td[2]").text
            Defendant1Address_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[2]/td[3]").text
            Defendant1Attorney_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[2]/td[5]").text

            if check_exists_by_xpath("/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[3]/td[1]") == True:
                Defendant2Name_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[3]/td[1]").text
                Defendant2DBATA_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[3]/td[2]").text
                Defendant2Address_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[3]/td[3]").text
                Defendant2Attorney_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[3]/td[5]").text

            if check_exists_by_xpath("/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[4]/td[1]") == True:
                Defendant3Name_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[4]/td[1]").text
                Defendant3DBATA_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[4]/td[2]").text
                Defendant3Address_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[4]/td[3]").text
                Defendant3Attorney_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[4]/td[5]").text

            if check_exists_by_xpath("/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[5]/td[1]") == True:
                Defendant4Name_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[5]/td[1]").text
                Defendant4DBATA_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[5]/td[2]").text
                Defendant4Address_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[5]/td[3]").text
                Defendant4Attorney_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[5]/td[5]").text

            if check_exists_by_xpath("/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[6]/td[1]") == True:
                Defendant5Name_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[6]/td[1]").text
                Defendant5DBATA_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[6]/td[2]").text
                Defendant5Address_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[6]/td[3]").text
                Defendant5Attorney_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[6]/td[5]").text

            if check_exists_by_xpath("/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[7]/td[1]") == True:
                Defendant6Name_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[7]/td[1]").text
                Defendant6DBATA_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[7]/td[2]").text
                Defendant6Address_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[7]/td[3]").text
                Defendant6Attorney_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[6]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[7]/td[5]").text
            # Need to collect next hearing date if continued (will consider no judgment a continuance) - NOTE: Can you use similar code for admin hearings
            # Find number of rows in the table 
            #rows = browser.find_elements_by_xpath("/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[8]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr")
            #row_num = len(rows)
            Next_Hearing_Date_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[8]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[2]/td[1]").text
            Next_Hearing_Time_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[8]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[2]/td[2]").text
            if browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[8]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[2]/td[4]").text == "Administrative Hearing":
                admin_tag = True
            else:
                admin_tag = False

        else:
            CaseNumber_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td[2]").text
            FiledDate_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td[4]").text
            CaseType_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td[2]").text
            DebtType_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td[4]").text
            Judgment_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[15]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td[2]").text
            Costs_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[15]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td[4]").text
            AttorneyFees_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[15]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td[6]").text
            PrincipalAmount_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[15]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td[2]").text
            OtherAmount_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[15]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td[4]").text
            InterestAward_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[15]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td[6]").text
            Possession_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[15]/td/table/tbody/tr[2]/td/table/tbody/tr[3]/td[2]").text
            # WritIssuedDate may be redundant with WritofEvictionIssuedDate - right now just pulling the same information into both
            WritIssuedDate_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[15]/td/table/tbody/tr[2]/td/table/tbody/tr[3]/td[4]").text
            HomesteadExemptionWaived_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[15]/td/table/tbody/tr[2]/td/table/tbody/tr[4]/td[2]").text
            IsJudgmentSatisfied_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[15]/td/table/tbody/tr[2]/td/table/tbody/tr[5]/td[2]").text
            DateSatisfactionFiled_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[15]/td/table/tbody/tr[2]/td/table/tbody/tr[5]/td[4]").text
            OtherAwarded_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[15]/td/table/tbody/tr[2]/td/table/tbody/tr[5]/td[6]").text
            FurtherCaseInformation_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[15]/td/table/tbody/tr[2]/td/table/tbody/tr[6]/td[2]").text
            Garnishee_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[17]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td[2]").text
            Address_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[17]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td[4]").text
            GarnisheeAnswer_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[17]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td[2]").text
            AnswerDate_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[17]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td[3]/table/tbody/tr/td[2]").text
            NumberofChecksReceived_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[17]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td[3]/table/tbody/tr/td[4]").text
            AppealDate_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[19]/td/table/tbody/tr[2]/td/table/tbody/tr/td[2]").text
            AppealedBy_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[19]/td/table/tbody/tr[2]/td/table/tbody/tr/td[4]").text
            WritofEvictionIssuedDate_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[15]/td/table/tbody/tr[2]/td/table/tbody/tr[3]/td[4]").text
            WritofFieriFaciasIssuedDate_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[15]/td/table/tbody/tr[2]/td/table/tbody/tr[3]/td[6]").text


            Plaintiff1Name_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[5]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[2]/td[1]").text
            Plaintiff1DBATA_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[5]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[2]/td[2]").text
            Plaintiff1Address_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[5]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[2]/td[3]").text
            Plaintiff1Attorney_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[5]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[2]/td[5]").text

            # Had to wrap additional plaintiffs and defendents in if statements to see if the information exists
            if check_exists_by_xpath("/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[5]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[3]/td[1]") == True:
                Plaintiff2Name_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[5]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[3]/td[1]").text
                Plaintiff2DBATA_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[5]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[3]/td[2]").text
                Plaintiff2Address_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[5]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[3]/td[3]").text
                Plaintiff2Attorney_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[5]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[3]/td[5]").text

            if check_exists_by_xpath("/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[5]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[4]/td[1]") == True:
                Plaintiff3Name_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[5]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[4]/td[1]").text
                Plaintiff3DBATA_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[5]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[4]/td[2]").text
                Plaintiff3Address_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[5]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[4]/td[3]").text
                Plaintiff3Attorney_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[5]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[4]/td[5]").text

            Defendant1Name_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[2]/td[1]").text
            Defendant1DBATA_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[2]/td[2]").text
            Defendant1Address_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[2]/td[3]").text
            Defendant1Attorney_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[2]/td[5]").text

            if check_exists_by_xpath("/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[3]/td[1]") == True:
                Defendant2Name_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[3]/td[1]").text
                Defendant2DBATA_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[3]/td[2]").text
                Defendant2Address_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[3]/td[3]").text
                Defendant2Attorney_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[3]/td[5]").text

            if check_exists_by_xpath("/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[4]/td[1]") == True:
                Defendant3Name_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[4]/td[1]").text
                Defendant3DBATA_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[4]/td[2]").text
                Defendant3Address_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[4]/td[3]").text
                Defendant3Attorney_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[4]/td[5]").text

            if check_exists_by_xpath("/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[5]/td[1]") == True:
                Defendant4Name_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[5]/td[1]").text
                Defendant4DBATA_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[5]/td[2]").text
                Defendant4Address_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[5]/td[3]").text
                Defendant4Attorney_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[5]/td[5]").text

            if check_exists_by_xpath("/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[6]/td[1]") == True:
                Defendant5Name_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[6]/td[1]").text
                Defendant5DBATA_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[6]/td[2]").text
                Defendant5Address_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[6]/td[3]").text
                Defendant5Attorney_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[6]/td[5]").text

            if check_exists_by_xpath("/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[7]/td[1]") == True:
                Defendant6Name_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[7]/td[1]").text
                Defendant6DBATA_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[7]/td[2]").text
                Defendant6Address_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[7]/td[3]").text
                Defendant6Attorney_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[7]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[7]/td[5]").text
            # Need to collect next hearing date if continued (will consider no judgment a continuance) - NOTE: Can you use similar code for admin hearings
            #rows = browser.find_elements_by_xpath("/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[9]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr")
            #row_num = len(rows)
            Next_Hearing_Date_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[9]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[2]/td[1]").text
            Next_Hearing_Time_i = browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[9]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[2]/td[2]").text
            if browser.find_element("xpath","/html/body/table[1]/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/form/div/table/tbody/tr[9]/td/table/tbody/tr[2]/td/table/tbody/tr/td/table/tbody/tr[2]/td[4]").text == "Administrative Hearing":
                admin_tag = True
            else:
                admin_tag = False


        CaseNumber.append(CaseNumber_i)
        FiledDate.append(FiledDate_i)
        CaseType.append(CaseType_i)
        DebtType.append(DebtType_i)
        Judgment.append(Judgment_i)
        Costs.append(Costs_i)
        AttorneyFees.append(AttorneyFees_i)
        PrincipalAmount.append(PrincipalAmount_i)
        OtherAmount.append(OtherAmount_i)
        InterestAward.append(InterestAward_i)
        Possession.append(Possession_i)
        WritIssuedDate.append(WritIssuedDate_i)
        HomesteadExemptionWaived.append(HomesteadExemptionWaived_i)
        IsJudgmentSatisfied.append(IsJudgmentSatisfied_i)
        DateSatisfactionFiled.append(DateSatisfactionFiled_i)
        OtherAwarded.append(OtherAwarded_i)
        FurtherCaseInformation.append(FurtherCaseInformation_i)
        Garnishee.append(Garnishee_i)
        Address.append(Address_i)
        GarnisheeAnswer.append(GarnisheeAnswer_i)
        AnswerDate.append(AnswerDate_i)
        NumberofChecksReceived.append(NumberofChecksReceived_i)
        AppealDate.append(AppealDate_i)
        AppealedBy.append(AppealedBy_i)
        WritofEvictionIssuedDate.append(WritofEvictionIssuedDate_i)
        WritofFieriFaciasIssuedDate.append(WritofFieriFaciasIssuedDate_i)
        Plaintiff1Name.append(Plaintiff1Name_i)
        Plaintiff1DBATA.append(Plaintiff1DBATA_i)
        Plaintiff1Address.append(Plaintiff1Address_i)
        Plaintiff1Attorney.append(Plaintiff1Attorney_i)
        Plaintiff2Name.append(Plaintiff2Name_i)
        Plaintiff2DBATA.append(Plaintiff2DBATA_i)
        Plaintiff2Address.append(Plaintiff2Address_i)
        Plaintiff2Attorney.append(Plaintiff2Attorney_i)
        Plaintiff3Name.append(Plaintiff3Name_i)
        Plaintiff3DBATA.append(Plaintiff3DBATA_i)
        Plaintiff3Address.append(Plaintiff3Address_i)
        Plaintiff3Attorney.append(Plaintiff3Attorney_i)
        Defendant1Name.append(Defendant1Name_i)
        Defendant1DBATA.append(Defendant1DBATA_i)
        Defendant1Address.append(Defendant1Address_i)
        Defendant1Attorney.append(Defendant1Attorney_i)
        Defendant2Name.append(Defendant2Name_i)
        Defendant2DBATA.append(Defendant2DBATA_i)
        Defendant2Address.append(Defendant2Address_i)
        Defendant2Attorney.append(Defendant2Attorney_i)
        Defendant3Name.append(Defendant3Name_i)
        Defendant3DBATA.append(Defendant3DBATA_i)
        Defendant3Address.append(Defendant3Address_i)
        Defendant3Attorney.append(Defendant3Attorney_i)
        Defendant4Name.append(Defendant4Name_i)
        Defendant4DBATA.append(Defendant4DBATA_i)
        Defendant4Address.append(Defendant4Address_i)
        Defendant4Attorney.append(Defendant4Attorney_i)
        Defendant5Name.append(Defendant5Name_i)
        Defendant5DBATA.append(Defendant5DBATA_i)
        Defendant5Address.append(Defendant5Address_i)
        Defendant5Attorney.append(Defendant5Attorney_i)
        Defendant6Name.append(Defendant6Name_i)
        Defendant6DBATA.append(Defendant6DBATA_i)
        Defendant6Address.append(Defendant6Address_i)
        Defendant6Attorney.append(Defendant6Attorney_i)
        Next_Hearing_Date.append(Next_Hearing_Date_i)
        Next_Hearing_Time.append(Next_Hearing_Time_i)
        Admin_tag.append(admin_tag)

In [10]:
# Create a df of the lists
d = {'CaseNumber':CaseNumber,
'FiledDate':FiledDate,
'CaseType':CaseType,
'DebtType':DebtType,
'Judgment':Judgment, 
'Costs':Costs,
'AttorneyFees':AttorneyFees, 
'PrincipalAmount':PrincipalAmount,
'OtherAmount':OtherAmount,
'InterestAward':InterestAward,
'Possession':Possession,
'WritIssuedDate':WritIssuedDate,
'HomesteadExemptionWaived':HomesteadExemptionWaived,
'IsJudgmentSatisfied': IsJudgmentSatisfied,
'DateSatisfactionFiled': DateSatisfactionFiled,
'OtherAwarded':OtherAwarded,
'FurtherCaseInformation':FurtherCaseInformation,
'Garnishee':Garnishee,
'Address': Address,
'GarnisheeAnswer': GarnisheeAnswer,
'AnswerDate': AnswerDate,
'NumberofChecksReceived':NumberofChecksReceived,
'AppealDate': AppealDate,
'AppealedBy':AppealedBy,
'WritofEvictionIssuedDate': WritofEvictionIssuedDate,
'WritofFieriFaciasIssuedDate':WritofFieriFaciasIssuedDate,
'Plaintiff1Name':Plaintiff1Name,
'Plaintiff1DBATA':Plaintiff1DBATA,
'Plaintiff1Address': Plaintiff1Address,
'Plaintiff1Attorney': Plaintiff1Attorney, 
'Plaintiff2Name': Plaintiff2Name,
'Plaintiff2DBATA':Plaintiff2DBATA, 
'Plaintiff2Address':Plaintiff2Address,
'Plaintiff2Attorney':Plaintiff2Attorney,
'Plaintiff3Name':Plaintiff3Name,
'Plaintiff3DBATA':Plaintiff3DBATA,
'Plaintiff3Address':Plaintiff3Address,
'Plaintiff3Attorney':Plaintiff3Attorney,
'Defendant1Name':Defendant1Name,
'Defendant1DBATA':Defendant1DBATA,
'Defendant1Address':Defendant1Address,
'Defendant1Attorney':Defendant1Attorney,
'Defendant2Name':Defendant2Name,
'Defendant2DBATA':Defendant2DBATA,
'Defendant2Address':Defendant2Address,
'Defendant2Attorney':Defendant2Attorney,
'Defendant3Name':Defendant3Name,
'Defendant3DBATA':Defendant3DBATA,
'Defendant3Address':Defendant3Address,
'Defendant3Attorney':Defendant3Attorney,
'Defendant4Name':Defendant4Name,
'Defendant4DBATA':Defendant4DBATA,
'Defendant4Address':Defendant4Address,
'Defendant4Attorney':Defendant4Attorney,
'Defendant5Name':Defendant5Name,
'Defendant5DBATA':Defendant5DBATA,
'Defendant5Address':Defendant5Address,
'Defendant5Attorney':Defendant5Attorney,
'Defendant6Name':Defendant6Name,
'Defendant6DBATA':Defendant6DBATA,
'Defendant6Address':Defendant6Address,
'Defendant6Attorney':Defendant6Attorney, 
'Next_Hearing_Date':Next_Hearing_Date,
'Next_Hearing_Time':Next_Hearing_Time, 
    'AdministrativeHearing':Admin_tag}

df_detail = pd.DataFrame(d)

TodaysDate = pd.to_datetime('today').date()
#TodaysDate = time.strftime("%d-%m-%Y")

df_detail['Date_Collected'] = TodaysDate

TodaysDate = time.strftime("%d-%m-%Y")

# Export to detail file
df_detail.to_csv('C:/Users/katherine.key/OneDrive - City of Alexandria/Documents/Evictions/Data/Clean/Detailed_Outcomes/scraper_detail_'+TodaysDate+".csv", index = False)

# Export summary to file
UD_days.to_csv('C:/Users/katherine.key/OneDrive - City of Alexandria/Documents/Evictions/Data/Clean/Schedule_Summary/scraper_summary_'+TodaysDate+".csv", index = False)

In [11]:
df_detail.tail()

,CaseNumber,FiledDate,CaseType,DebtType,Judgment,Costs,AttorneyFees,PrincipalAmount,OtherAmount,InterestAward,...,Defendant5Address,Defendant5Attorney,Defendant6Name,Defendant6DBATA,Defendant6Address,Defendant6Attorney,Next_Hearing_Date,Next_Hearing_Time,AdministrativeHearing,Date_Collected
1303,GV25006832-00,09/12/2025,Unlawful Detainer,,,,,,,,...,,,,,,,09/30/2025,01:30 PM,False,2025-09-13
1304,GV25006833-00,09/12/2025,Unlawful Detainer,,,,,,,,...,,,,,,,09/30/2025,01:30 PM,False,2025-09-13
1305,GV25006835-00,09/12/2025,Unlawful Detainer,,,,,,,,...,,,,,,,09/30/2025,01:30 PM,False,2025-09-13
1306,GV25006836-00,09/12/2025,Unlawful Detainer,,,,,,,,...,,,,,,,09/30/2025,01:30 PM,False,2025-09-13
1307,GV25006838-00,09/12/2025,Unlawful Detainer,,,,,,,,...,,,,,,,09/30/2025,01:30 PM,False,2025-09-13
